In [3]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

import lightgbm as lgb

%matplotlib inline

try:
    # To enable interactive mode you should install ipywidgets
    # https://github.com/jupyter-widgets/ipywidgets
    from ipywidgets import interact, SelectMultiple
    INTERACTIVE = True
except ImportError:
    INTERACTIVE = False

In [8]:
import mlflow
mlflow.set_tracking_uri("http://mlflow_server:5000")
project_uri = "https://github.com/mlflow/mlflow-example"
params = {"alpha": 0.5, "l1_ratio": 0.01}

# Run MLflow project and create a reproducible conda environment
# on a local host
mlflow.run(project_uri, parameters=params,user_id='eu')


response = mlflow.projects.run(project_uri, parameters=params,use_conda=False)

TypeError: run() got an unexpected keyword argument 'user_id'

In [31]:
df_train = pd.read_csv(str( 'sample_products.csv'),sep=',')
df_test = pd.read_csv(str( 'test_products.csv'), sep=',')

*Limpar dados (excluir linhas com dados em colunas faltando ou excluir colunas inteiras)*

In [32]:
df_train.head()
df_train = df_train.drop('order_counts', axis=1)
df_train.isnull().sum().sort_values(ascending=False)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38000 entries, 0 to 37999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         38000 non-null  int64  
 1   seller_id          38000 non-null  int64  
 2   query              38000 non-null  object 
 3   search_page        38000 non-null  int64  
 4   position           38000 non-null  int64  
 5   title              38000 non-null  object 
 6   concatenated_tags  37998 non-null  object 
 7   creation_date      38000 non-null  object 
 8   price              38000 non-null  float64
 9   weight             37942 non-null  float64
 10  express_delivery   38000 non-null  int64  
 11  minimum_quantity   38000 non-null  int64  
 12  view_counts        38000 non-null  int64  
 13  category           38000 non-null  object 
dtypes: float64(2), int64(7), object(5)
memory usage: 4.1+ MB


In [34]:
df_train['query']

0                                  espirito santo
1                                cartao de visita
2                           expositor de esmaltes
3             medidas lencol para berco americano
4                            adesivo box banheiro
                           ...                   
37995    mochila personalizada galinha pintadinha
37996                            tag dia dos pais
37997                       kit bolsa maternidade
37998                               festa 15 anos
37999               painel aniversariantes em eva
Name: query, Length: 38000, dtype: object

*Separar conjunto de treinamento, teste e validação*

In [33]:
y_train = df_train['category']
y_test = df_test['category']
X_train = df_train.drop('category', axis=1)
X_test = df_test.drop('category', axis=1)

*Criar coluna para cada categoria (separadamente)*

*Encodar colunas string em 

In [18]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [26]:
params = {
    'metric': 'binary_logloss',
    'verbose': -1,
    'application': 'binary',
    'device' : 'cpu'
}

In [28]:
evals_result = {}  # to record eval results for plotting
clf = lgb.LGBMClassifier()